# 二項係数 $\binom{2n}{n}$ を $n$ で割った余りについて 〜 "エレガントな解答求む"より〜

## モジュール

In [1]:
import math
from decimal import *
getcontext().prec = (10**3)*5
from fractions import Fraction

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

from functools import reduce

## 二項係数

### 二項係数を返す関数の2種の定義

In [20]:
def binom(n,k):
    return Decimal(math.factorial(n))/Decimal(math.factorial(k)*math.factorial(n-k))

def binom01(n,k):
    if k==0:
        nume, deno = 1, 1
    else:
        nume = reduce(lambda x,y: x*y, range(n-k+1, n+1))
        deno = reduce(lambda x,y: x*y, range(1, k+1))
    return Decimal(nume/deno)

def binom02(n,k):
    rtn = 1
    for i in range(k):
        rtn = Fraction(rtn.numerator * (n-i), rtn.denominator * (k-i))
#         rtn = rtn * Fraction(n-i, k-i)  # <-- だいぶ遅い
    if rtn.denominator == 1:
        return Decimal(rtn.numerator)
    else:
        return None

In [22]:
num = 20
#print([binom(num, k) for k in range(0,num+1)])
print([binom01(num, k) for k in range(0,num+1)])
#print([binom02(num, k) for k in range(0,num+1)])

[Decimal('1'), Decimal('20'), Decimal('190'), Decimal('1140'), Decimal('4845'), Decimal('15504'), Decimal('38760'), Decimal('77520'), Decimal('125970'), Decimal('167960'), Decimal('184756'), Decimal('167960'), Decimal('125970'), Decimal('77520'), Decimal('38760'), Decimal('15504'), Decimal('4845'), Decimal('1140'), Decimal('190'), Decimal('20'), Decimal('1')]


### 2種の関数の速さ比べ

In [23]:
N = 10**2
%timeit [binom(2*n, n) for n in range(1,N)]
%timeit [binom01(2*n, n) for n in range(1,N)]
%timeit [binom02(2*n, n) for n in range(1,N)]

100 loops, best of 3: 2.77 ms per loop
1000 loops, best of 3: 1.22 ms per loop
100 loops, best of 3: 10.9 ms per loop


In [ ]:
X = [binom01(2*n, n) for n in tqdm(range(1,N))]

In [ ]:
Y = [binom02(2*n, n) for n in tqdm(range(1,N))]

In [ ]:
X==Y

## グラフ描画

### $n = 1, 2, \dots, 2000$ において

In [ ]:
N = (10**3)*2

X = range(1,N+1)
Y = [binom01(2*n,n)%n for n in tqdm(X)]

In [ ]:
plt.figure(figsize = (16,7))

plt.subplot(121)
plt.plot(X, Y, 'b.', ms = 1.6)
plt.subplot(122)
plt.plot(X, Y, 'b.', ms = 1.6)
plt.yscale('symlog')

plt.show()

### `pickle` モジュール (データの永続化) の利用

#### pickle module をインポート

In [ ]:
import pickle

#### データをピックル化

In [ ]:
import os.path

N = 10**4
getcontext().prec = N

filename = 'data_'+str(N)+'.pickle'

if os.path.exists(filename):
    print('{} exists'.format(filename))
else:
    data = {
        'range': range(1,N+1),
        'values': [int(binom02(2*n,n)%n) for n in tqdm(range(1,N+1))]
        }
    with open(filename, 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

#### ピックル化されたデータを読み込み描画

In [ ]:
# pickle 化されたデータを読み込みます。

filename = 'data_10000.pickle'
with open(filename, 'rb') as f:
    # The protocol version used is detected automatically, so we do not have to specify it.
    data = pickle.load(f)

In [ ]:
N = 10000 # 5000 # 3000 # 1000 #

X = range(1, N+1)
Y = data['values'][:N]

plt.figure(figsize = (16, 7))

plt.subplot(121)
plt.plot(X,Y, 'b.', ms = 0.5)
plt.subplot(122)
plt.plot(X,Y, 'b.', ms = 0.5)
plt.yscale('symlog')
    
plt.show()

## 余りとして現れる数値の頻度

In [ ]:
N = 5000 # 2000 # 10000 # 3000

with open('data_10000.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not have to specify it.
    data = pickle.load(f)

In [ ]:
X = data['range']
cnt = [data['values'].count(n) for n in X]

In [ ]:
plt.figure(figsize = (16,7))

plt.subplot(121)
plt.plot(X, data['values'], 'b.', ms = 0.5)
plt.title('f(n) = 2n_C_n (mod n)')
plt.subplot(122)
# plt.plot(X, cnt, lw = 0.5)
plt.plot(X, cnt, 'b.')
plt.title('histgram of remainders')
plt.xscale('log')
# plt.yscale('log')

plt.show()

In [ ]:
vlist = {num: data['values'][:2000].count(num) for num in X}.items()
sorted(vlist, key=lambda vlist: vlist[1], reverse=True)[0:10]

In [ ]:
data['values'][:2001].count(0)

In [ ]:
print([k + 1 for k in range(1,101) if data['values'][k] == 0])

In [ ]:
Z = [binom01(2*n,n)%n for n in tqdm(range(1,2000))]

In [ ]:
[int(d) for d in Z].count(0)